In [1]:
from google.colab import drive
drive.mount('/gdrive')

%cd "/gdrive/My Drive/Day_Night"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive/My Drive/Day_Night


In [2]:
from keras.applications import VGG16
from keras.optimizers import Adam, SGD
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Activation,GlobalMaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
import os
import cv2
from shutil import copyfile as cp

Using TensorFlow backend.


In [0]:
def readImg(path):
  img = cv2.imread(path)
  img = cv2.resize(img, (224,224), interpolation = cv2.INTER_LANCZOS4)
  img = img.reshape((1,224, 224, 3))
  return img

In [0]:
def test(path):
  if not os.path.exists("result"):
    os.makedirs("result/day")
    os.makedirs("result/night")
    img_list = os.listdir(path)
    for imgName in img_list:
      print(imgName)
      img = readImg(path+"/"+imgName)
      res = model.predict(img)
      if res[0][0] > res[0][1]:
        cp(path+"/"+imgName,"result/day/"+imgName)
      else:
        cp(path+"/"+imgName,"result/night/"+imgName)

In [0]:
def train(path="/", learn=False):
  if learn == False:
    vgg_conv = load_model("day_night.h5")
  else:
    vgg_conv = VGG16(weights='imagenet',
                      include_top=False,
                      input_shape=(224, 224, 3))
  pre_trained_model = vgg_conv
  for layer in pre_trained_model.layers[:15]:
      layer.trainable = False
  for layer in pre_trained_model.layers[15:]:
      layer.trainable = True

  last_layer = pre_trained_model.get_layer('block5_pool')
  last_output = last_layer.output

  x = GlobalMaxPooling2D()(last_output)
  x = Dense(512, activation="sigmoid")(x) 
  x = Dropout(0.5)(x)
  x = Dense(2, activation="softmax")(x)
  opt = SGD(lr=0.00001)
  model = Model(pre_trained_model.input, x)
  model.compile(loss='categorical_crossentropy',
                optimizer=opt,
                metrics=['accuracy'])
  train_datagen = ImageDataGenerator(
      fill_mode='nearest'
  )
  train_generator = train_datagen.flow_from_directory(
      path,
      target_size=(224, 224),
      batch_size = 5,
      shuffle=True,
      class_mode='categorical')

  history = model.fit_generator(
      train_generator,
      epochs=40,
      steps_per_epoch=train_generator.n//5)

In [6]:
train("images/train")

Found 240 images belonging to 2 classes.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/40
48/48 [==============================] - 108s 2s/step - loss: 0.8305 - acc: 0.5292
Epoch 2/40
48/48 [==============================] - 5s 110ms/step - loss: 0.8121 - acc: 0.5625
Epoch 3/40
48/48 [==============================] - 5s 110ms/step - loss: 0.7378 - acc: 0.6167
Epoch 4/40
48/48 [==============================] - 5s 110ms/step - loss: 0.6764 - acc: 0.6667
Epoch 5/40
48/48 [==============================] - 5s 111ms/step - loss: 0.6736 - acc: 0.6292
Epoch 6/40
48/48 [==============================] - 5s 112ms/step - loss: 0.5762 - acc: 0.7292
Epoch 7/40
48/48 [==============================] - 5s 111ms/step - loss: 0.5159 - acc: 0.7542
Epoch 8/40
48/48 [==============================] - 5s 112ms/step - loss: 0.4768 - acc: 0.7625
Epoch 9/40
48/48 [==============================] - 5s 113ms/step - loss: 0.5241 - acc: 0.7542
Epoch 10/

In [0]:
test("images/test")